1. Longest Increasing Subsequence
2. Increasing Triplet Subsequence

---

# 1. Longest Increasing Subsequence

The goal is to find the length of the LIS.

*You have already solved this before.* 

Maintain an array which tracks elements -- your job is to maintain this array in an increasing manner.

Given `3, 5, 1, 7`, your array will evolve like so:
* `[3]`
* `[3, 5]` because `5` is greater than the last element
* `[1, 5]` -- this is tricky:
    * `1` is clearly smaller than both, but you donot want to discard the current sequence, because `7` appears next.
    * Instead, you `swap` the number in the right place -- this works because ANY number smaller than 3, could have been in 3's place, and the size of the LIS would not change.
* `[1, 5, 7]` -- again, this is NOT the correct subsequence, but the size is correct.

Now, say you actually wanted to track the Longest Subsequence, and not just the size?

# 2. Increasing Triplet Subsequence

An extension of LIS -- does the array contain three indices `i, j, k` such that `array[i] < array[j] < array[k]`.

In [1]:
class IncreasingTriplet:
    def solve(self, nums):
        triplet = []
        for ix, n in enumerate(nums):
            if ix == 0:
                triplet.append(n)
            else:
                if n > triplet[-1]:
                    triplet.append(n)
                else:
                    # It can only have two elements, of which you need to replace
                    # the first, or the 2nd.
                    if n > triplet[0]:
                        triplet[1] = n
                    else:
                        triplet[0] = n
                    
            if len(triplet) == 3:
                return True
            
        return False

In [5]:
o = IncreasingTriplet()
print(o.solve([1,2,3,4,5]))
print(o.solve([5, 4, 3, 2, 1]))
print(o.solve([2,1,5,0,4,6]))

True
False
True
